# Stock Price Direction Prediction - Dataset Generation

This notebook generates two datasets for predicting stock price direction:
1. **Basic Dataset**: Raw OHLCV data with binary target
2. **Enhanced Dataset**: OHLCV data + technical indicators with binary target

## Setup and Data Collection
- Tickers: Top 50 S&P 500 stocks by market cap
- Date Range: 2010-01-01 to 2024-12-31
- Train/Test Split: 2010-2021 (Train), 2022-2024 (Test)

In [6]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import time
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


Pandas methods used in this notebook — quick reference

Below are concise, actionable descriptions of the DataFrame/Series methods and operations that appear in this notebook. Each item shows what it does and a brief example of the effect.

- shift(n)
  - Moves values up/down along the index. Commonly used to compare current row with next/previous row.
  - Example: df['Close'].shift(-1) gives next-day close.

- astype(type)
  - Casts dtype of a Series (or DataFrame) to the given type.
  - Example: (next_close > df['Close']).astype(int) converts boolean to 0/1.

- df.columns (assignment)
  - Replace column labels. Used here to flatten MultiIndex columns by joining parts or taking the last part.
  - Example: df.columns = [f"{p}_{t}" for p,t in df.columns]

- isinstance(df.columns, pd.MultiIndex)
  - Checks whether columns have multiple levels (e.g., (Price, Ticker)). If true, you may want to flatten or stack.

- get_level_values(level)
  - Returns the labels for a specific MultiIndex level (useful to find empty tickers or particular level values).
  - Example: df.columns.get_level_values('Ticker')

- stack(level)
  - Moves one column-level into the row index, turning wide MultiIndex columns into long rows. Use when you want a tidy table with a Ticker column.
  - Example: df.stack(level='Ticker') → index becomes (original_index, Ticker) and price labels stay as columns.

- reset_index()
  - Converts index levels into columns and resets index to default integer index.
  - Often used after stack() to get a flat DataFrame: df.stack(...).reset_index()

- sort_values(by)
  - Sorts rows by one or more columns. Important before groupby operations that assume ordering.
  - Example: df.sort_values(['Ticker','Date'])

- reset_index(drop=True)
  - Reset index and drop the old index instead of adding it as a column.

- groupby(...).apply(func)
  - Group rows and apply a function to each group. Useful for per-ticker operations, e.g., remove last row for each ticker: df.groupby('Ticker').apply(lambda x: x.iloc[:-1])

- iloc / loc
  - iloc: integer-location based selection. loc: label-based selection. Both used to select rows/columns precisely.
  - Example: x.iloc[:-1] takes all but the last row of group x. dataset2.loc[mask, 'SMA_10'] assigns values to rows matching mask.

- unique()
  - Returns unique values in a Series (e.g., list of tickers).

- isnull().sum()
  - Counts missing values per column.

- dropna()
  - Drops rows (or columns) containing NA. Here used to remove rows missing indicator values after rolling/ewm computations.

- rolling(window).mean()
  - Computes rolling statistics (SMA) over a window of rows.
  - Example: df['Close'].rolling(10).mean() gives 10-day SMA.

- diff()
  - First discrete difference on Series: current - previous. Used to compute gains/losses for RSI.

- where(condition, other)
  - Keeps values where condition True, else replaces with other. Useful to separate gains (positive diffs) from losses.

- ewm(span, adjust=False).mean()
  - Exponential weighted mean (EMA). Used for MACD and signal smoothing.

- drop_duplicates()
  - Remove duplicate rows (not used heavily here but commonly used when combining datasets).

- to_csv / to_parquet
  - Persist DataFrame to disk in CSV or Parquet formats.

- melt(id_vars, var_name, value_name)
  - Unpivot a DataFrame from wide to long format. Use when you flattened column names like 'Close_AAPL' and want rows per ticker.
  - Example: df.reset_index().melt(id_vars='Date')

- str.rsplit(sep, n=1, expand=True)
  - Split string column from right into parts. Useful to split 'Close_AAPL' into ['Close','AAPL'].

- copy()
  - Create an explicit copy of a DataFrame to avoid modifying the original view in-place.

- head(n) / shape / columns / value_counts() / mean()
  - Utility methods: head shows top rows, shape gives (rows, cols), columns lists column names, value_counts gives counts per value, mean gives average (useful for target balance).

Tip: for tidy machine learning datasets prefer the long form (one row per Date+Ticker) produced by stacking or melting — columns: Date, Ticker, Open, High, Low, Close, Volume, indicators..., Target.

## Step 1: Get Top 50 S&P 500 Stocks by Market Cap

In [8]:
import yfinance as yf
import pandas as pd

# List of top 50 tickers
top_50_tickers = [
    'AAPL','MSFT','AMZN','GOOGL','BRK-B','JPM','JNJ','PG','V','UNH'
]

# Fix for Yahoo Finance tickers with dashes
top_50_tickers = [ticker.replace('-', '-') for ticker in top_50_tickers]

# Download data for all tickers
all_data = []

for ticker in top_50_tickers:
    try:
        print(f"Downloading {ticker}...")
        df = yf.download(ticker, start="2020-01-01", end="2025-11-12")
        df['Ticker'] = ticker
        all_data.append(df)
    except Exception as e:
        print(f"Failed for {ticker}: {e}")

# Combine into one DataFrame
combined_df = pd.concat(all_data)
combined_df.reset_index(inplace=True)

print("✅ Data download complete.")
print("Shape:", combined_df.shape)
print(combined_df.head())


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

✅ Data download complete.
Shape: (14740, 52)
Price        Date      Close       High        Low       Open       Volume  \
Ticker                  AAPL       AAPL       AAPL       AAPL         AAPL   
0      2020-01-02  72.468254  72.528574  71.223252  71.476592  135480400.0   
1      2020-01-03  71.763733  72.523762  71.539345  71.696175  146322800.0   
2      2020-01-06  72.335564  72.374169  70.634547  70.885479  118387200.0   
3      2020-01-07  71.995361  72.600968  71.775796  72.345212  108872000.0   
4      2020-01-08  73.153488  73.455087  71.698574  71.698574  132079200.0   

Price  Ticker Close High  Low  ... Close High Low Open Volume Close High Low  \
Ticker         MSFT MSFT MSFT  ...     V    V   V    V      V   UNH  UNH UNH   
0        AAPL   NaN  NaN  NaN  ...   NaN  NaN NaN  NaN    NaN   NaN  NaN NaN   
1        AAPL   NaN  NaN  NaN  ...   NaN  NaN NaN  NaN    NaN   NaN  NaN NaN   
2        AAPL   NaN  NaN  NaN  ...   NaN  NaN NaN  NaN    NaN   NaN  NaN NaN   
3       

In [3]:
# ...existing code...
# Replace Wikipedia scraping with a fixed list of 10 tickers public since 2010

# ...existing code...

## Step 2: Download Historical Data

In [9]:
# Define date ranges
start_date = '2010-01-01'
end_date = '2024-12-31'

print(f"Downloading historical data from {start_date} to {end_date}...\n")

# Download data for all tickers
all_data = []

if not top_50_tickers:
    raise RuntimeError("No tickers available to download. Check the Wikipedia scraping step.")

for ticker in top_50_tickers:
    try:
        print(f"Downloading {ticker}...", end=' ')
        
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        
        if not df.empty:
            next_close = df['Close'].shift(-1)
            df['Target'] = (next_close > df['Close']).astype(int)
            
            
            all_data.append(df)
            print(f"OK ({len(df)} rows)")
            
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel('Ticker')
                print(df.columns)
        else:
            print("No data available")
    except Exception as e:
        print("Error:", e)
    time.sleep(0.5)  # be polite to the data provider



Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')
Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Target'], dtype='object', name='Price')


In [10]:

# Combine all data
if not all_data:
    raise RuntimeError("No data was downloaded for any tickers. Aborting.")


raw_data = all_data
print('raw_data ---------------------------------------------')
print(raw_data[0])


raw_data ---------------------------------------------
Price            Close        High         Low        Open     Volume  Target
Date                                                                         
2010-01-04    6.418384    6.433080    6.369499    6.400989  493729600       1
2010-01-05    6.429481    6.465770    6.395590    6.436079  601904800       0
2010-01-06    6.327211    6.454973    6.320613    6.429481  552160000       0
2010-01-07    6.315513    6.358101    6.269627    6.350603  477131200       1
2010-01-08    6.357502    6.358103    6.269929    6.307118  447610800       0
...                ...         ...         ...         ...        ...     ...
2024-12-23  254.120697  254.498976  252.308884  253.622948   40858800       1
2024-12-24  257.037476  257.047410  254.140559  254.339671   23234700       1
2024-12-26  257.853760  258.928914  256.470034  257.027510   27237100       0
2024-12-27  254.439224  257.535238  251.920617  256.669129   42355300       0
2024-12-3

## Step 4: Create Dataset 1 - Basic OHLCV Data

In [11]:
# Dataset 1: Basic OHLCV + Target
dataset1 = [data.copy(deep=True) for data in raw_data]
print(f"Dataset 1 (Basic) created")
print(dataset1[0].head(10))

Dataset 1 (Basic) created
Price          Close      High       Low      Open     Volume  Target
Date                                                                 
2010-01-04  6.418384  6.433080  6.369499  6.400989  493729600       1
2010-01-05  6.429481  6.465770  6.395590  6.436079  601904800       0
2010-01-06  6.327211  6.454973  6.320613  6.429481  552160000       0
2010-01-07  6.315513  6.358101  6.269627  6.350603  477131200       1
2010-01-08  6.357502  6.358103  6.269929  6.307118  447610800       0
2010-01-11  6.301419  6.388092  6.251633  6.382094  462229600       0
2010-01-12  6.229738  6.291220  6.190750  6.273825  594459600       1
2010-01-13  6.317612  6.326010  6.121172  6.234238  605892000       0
2010-01-14  6.281024  6.311916  6.268728  6.301419  432894000       0
2010-01-15  6.176055  6.346105  6.174256  6.326010  594067600       1


## Step 5: Calculate Technical Indicators for Dataset 2

Technical indicators to calculate:
- **Simple Moving Averages (SMA)**: 10-day, 50-day, 200-day
- **Relative Strength Index (RSI)**: 14-day period
- **MACD**: 12, 26, 9 parameters

In [7]:
def calculate_sma(data, window):
    """Calculate Simple Moving Average"""
    return data.rolling(window=window).mean()

def calculate_rsi(data, window=14):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(data, fast=12, slow=26, signal=9):
    """Calculate MACD and Signal line"""
    ema_fast = data.ewm(span=fast, adjust=False).mean()
    ema_slow = data.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

print("Technical indicator functions defined")

Technical indicator functions defined


In [8]:
# Create Dataset 2 with technical indicators
dataset2 = [data.copy(deep=True) for data in raw_data]

print("Calculating technical indicators for each ticker...\n")

for i in range(len(dataset2)):
    ticker_data = dataset2[i]
    ticker_data['SMA-10'] = calculate_sma(ticker_data['Close'], 10)
    ticker_data['SMA-50'] = calculate_sma(ticker_data['Close'], 50)
    ticker_data['SMA-200'] = calculate_sma(ticker_data['Close'], 200)
    
    ticker_data['RSI'] = calculate_rsi(ticker_data['Close'])
    
    ticker_data['MACD'], ticker_data['Signal_Line'] = calculate_macd(ticker_data['Close'])

    dataset2[i] = ticker_data.dropna()
    
    print(dataset2[i].head(5))
    

    

Calculating technical indicators for each ticker...

Price          Close      High       Low      Open      Volume  Target  \
Date                                                                     
2010-10-18  9.537151  9.567141  9.425884  9.551247  1093010800       0   
2010-10-19  9.281930  9.410291  8.997915  9.099285  1232784000       1   
2010-10-20  9.313118  9.424685  9.203351  9.267232   721624400       0   
2010-10-21  9.282829  9.439382  9.201253  9.368003   551460000       0   
2010-10-22  9.221343  9.298420  9.186254  9.269328   372778000       1   

Price         SMA-10    SMA-50   SMA-200        RSI      MACD  Signal_Line  
Date                                                                        
2010-10-18  8.968852  8.111438  7.362248  82.037181  0.321097     0.263761  
2010-10-19  9.030484  8.140073  7.376566  69.540844  0.321368     0.275283  
2010-10-20  9.094485  8.170736  7.390984  74.787042  0.320406     0.284307  
2010-10-21  9.155367  8.206324  7.405762  7

## Step 6: Clean Data - Drop Rows with Missing Values

## Step 7: Split Data into Train and Test Sets

- **Training**: 2010-2021
- **Testing**: 2022-2024

In [9]:
# Define split date
split_date = '2022-01-01'

# Combine the DFs

dataset1_full = pd.concat(dataset1)


dataset2_full = pd.concat(dataset2)

print(dataset1_full.head(5))
print(dataset2_full.head(5))

# Split Dataset 1

dataset1_train = dataset1_full[dataset1_full.index < split_date].copy()
dataset1_test = dataset1_full[dataset1_full.index >= split_date].copy()

# Split Dataset 2
dataset2_train = dataset2_full[dataset2_full.index < split_date].copy()
dataset2_test = dataset2_full[dataset2_full.index >= split_date].copy()

print(f"{'='*60}")
print(f"Dataset 1 (Basic OHLCV)")
print(f"{'='*60}")
print(f"Training set: {dataset1_train.shape[0]:,} rows ({dataset1_train.index.min()} to {dataset1_train.index.max()})")
print(f"Testing set:  {dataset1_test.shape[0]:,} rows ({dataset1_test.index.min()} to {dataset1_test.index.max()})")
print(f"\nTarget distribution - Train: {dataset1_train['Target'].mean()*100:.2f}% positive")
print(f"Target distribution - Test:  {dataset1_test['Target'].mean()*100:.2f}% positive")

print(f"\n{'='*60}")
print(f"Dataset 2 (With Technical Indicators)")
print(f"{'='*60}")
print(f"Training set: {dataset2_train.shape[0]:,} rows ({dataset2_train.index.min()} to {dataset2_train.index.max()})")
print(f"Testing set:  {dataset2_test.shape[0]:,} rows ({dataset2_test.index.min()} to {dataset2_test.index.max()})")
print(f"\nTarget distribution - Train: {dataset2_train['Target'].mean()*100:.2f}% positive")
print(f"Target distribution - Test:  {dataset2_test['Target'].mean()*100:.2f}% positive")

Price          Close      High       Low      Open     Volume  Target
Date                                                                 
2010-01-04  6.418384  6.433080  6.369499  6.400989  493729600       1
2010-01-05  6.429480  6.465769  6.395590  6.436078  601904800       0
2010-01-06  6.327211  6.454973  6.320613  6.429481  552160000       0
2010-01-07  6.315513  6.358101  6.269627  6.350603  477131200       1
2010-01-08  6.357501  6.358101  6.269928  6.307116  447610800       0
Price          Close      High       Low      Open      Volume  Target  \
Date                                                                     
2010-10-18  9.537151  9.567141  9.425884  9.551247  1093010800       0   
2010-10-19  9.281930  9.410291  8.997915  9.099285  1232784000       1   
2010-10-20  9.313118  9.424685  9.203351  9.267232   721624400       0   
2010-10-21  9.282829  9.439382  9.201253  9.368003   551460000       0   
2010-10-22  9.221343  9.298420  9.186254  9.269328   372778000    

## Step 8: Save Datasets to CSV and Parquet

In [10]:
# Create data directory if it doesn't exist
import os
os.makedirs('../data', exist_ok=True)

# Save Dataset 1 (Basic)
print("Saving Dataset 1 (Basic OHLCV)...")
dataset1_train.to_csv('../data/dataset1_train.csv', index=False)
dataset1_test.to_csv('../data/dataset1_test.csv', index=False)
dataset1_train.to_parquet('../data/dataset1_train.parquet', index=False)
dataset1_test.to_parquet('../data/dataset1_test.parquet', index=False)
print("âœ“ Dataset 1 saved")

# Save Dataset 2 (Enhanced)
print("\nSaving Dataset 2 (With Technical Indicators)...")
dataset2_train.to_csv('../data/dataset2_train.csv', index=False)
dataset2_test.to_csv('../data/dataset2_test.csv', index=False)
dataset2_train.to_parquet('../data/dataset2_train.parquet', index=False)
dataset2_test.to_parquet('../data/dataset2_test.parquet', index=False)
print("âœ“ Dataset 2 saved")

print(f"\n{'='*60}")
print("All datasets saved successfully!")
print(f"{'='*60}")
print("\nFiles created in ../data/ directory:")
print("  - dataset1_train.csv / .parquet")
print("  - dataset1_test.csv / .parquet")
print("  - dataset2_train.csv / .parquet")
print("  - dataset2_test.csv / .parquet")

Saving Dataset 1 (Basic OHLCV)...


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

## Step 9: Summary Statistics

In [ ]:
print("="*80)
print("DATASET GENERATION SUMMARY")
print("="*80)

print("\n1. DATA COLLECTION")
print(f"   - Number of stocks: {len(top_50_tickers)}")
print(f"   - Date range: 2010-01-01 to 2024-12-31")
print(f"   - Tickers: {', '.join(top_50_tickers[:10])}...")

print("\n2. DATASET 1 (Basic OHLCV)")
print(f"   Columns: {list(dataset1_train.columns)}")
print(f"   - Training: {dataset1_train.shape[0]:,} rows, {dataset1_train.shape[1]} features")
print(f"   - Testing:  {dataset1_test.shape[0]:,} rows, {dataset1_test.shape[1]} features")

print("\n3. DATASET 2 (With Technical Indicators)")
print(f"   Columns: {list(dataset2_train.columns)}")
print(f"   - Training: {dataset2_train.shape[0]:,} rows, {dataset2_train.shape[1]} features")
print(f"   - Testing:  {dataset2_test.shape[0]:,} rows, {dataset2_test.shape[1]} features")

print("\n4. TECHNICAL INDICATORS INCLUDED")
print("   - SMA_10, SMA_50, SMA_200 (Simple Moving Averages)")
print("   - RSI_14 (Relative Strength Index)")
print("   - MACD, MACD_Signal, MACD_Histogram")

print("\n5. FILE OUTPUTS")
print("   CSV and Parquet formats saved to ../data/ directory")

print("\n" + "="*80)
print("Dataset generation complete!")
print("="*80)

DATASET GENERATION SUMMARY

1. DATA COLLECTION
   - Number of stocks: 2
   - Date range: 2010-01-01 to 2024-12-31
   - Tickers: AAPL, MSFT...

2. DATASET 1 (Basic OHLCV)
   Columns: ['Close', 'High', 'Low', 'Open', 'Volume', 'Target']
   - Training: 6,042 rows, 6 features
   - Testing:  1,504 rows, 6 features

3. DATASET 2 (With Technical Indicators)
   Columns: ['Close', 'High', 'Low', 'Open', 'Volume', 'Target', 'SMA-10', 'SMA-50', 'SMA-200', 'RSI', 'MACD', 'Signal_Line']
   - Training: 5,644 rows, 12 features
   - Testing:  1,504 rows, 12 features

4. TECHNICAL INDICATORS INCLUDED
   - SMA_10, SMA_50, SMA_200 (Simple Moving Averages)
   - RSI_14 (Relative Strength Index)
   - MACD, MACD_Signal, MACD_Histogram

5. FILE OUTPUTS
   CSV and Parquet formats saved to ../data/ directory

Dataset generation complete!
